In [22]:
import pandas as pd
import numpy as np

In [23]:
econ_df = pd.read_csv("data/main/economic_data_with_ffq.csv", index_col="DATE", parse_dates=True)

econ_df.head()

,GDP,CPIAUCNS,UNRATE,FEDFUNDS,DTWEXM,T10Y2Y,M2,PCE,HOANBS,BUSINV,...,UMCSENT,OILPRICE,BAMLH0A0HYM2,GS10,CP,STLFSI,USSLIND,GDPC1,USROA,FFQ
DATE,,,,,,,,,,,,,,,,,,,,,
1997-01-03,8362.655000,159.1,5.30,5.250,88.83620,0.550,3820.0,5411.10,98.153000,1007785.0,...,97.40,25.170,3.093333,6.580,546.764000,0.728,1.610,11291.665000,1.270000,0.205587
1997-01-10,8374.668077,159.2,5.28,5.238,89.02160,0.580,3830.4,5415.68,98.189538,1008371.2,...,97.86,24.578,3.112000,6.548,548.187077,0.774,1.652,11306.100769,1.273077,0.203694
1997-01-17,8386.681154,159.3,5.26,5.226,89.46776,0.558,3833.9,5420.26,98.226077,1008957.4,...,98.32,23.986,3.052000,6.516,549.610154,0.732,1.694,11320.536538,1.276154,0.201802
1997-01-24,8398.694231,159.4,5.24,5.214,90.67085,0.550,3834.9,5424.84,98.262615,1009543.6,...,98.78,23.394,2.996000,6.484,551.033231,0.715,1.736,11334.972308,1.279231,0.199910
1997-01-31,8410.707308,159.5,5.22,5.202,91.54040,0.592,3836.7,5429.42,98.299154,1010129.8,...,99.24,22.802,2.954000,6.452,552.456308,0.747,1.778,11349.408077,1.282308,0.198018


In [24]:
import yfinance as yf

start = econ_df.index.min()
end = econ_df.index.max()

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

ticker_list = pd.read_html(url)[0]["Symbol"]
ticker_list = [i.replace('.', '-') for i in ticker_list]

stock_prices = yf.download(ticker_list, start, end)["Adj Close"]

stock_prices.head()

[*********************100%***********************]  503 of 503 completed


,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1997-01-03,NaN,NaN,0.164833,NaN,6.379129,2.083333,NaN,4.795903,8.594171,9.768930,...,NaN,7.310248,10.919062,6.595023,NaN,NaN,NaN,10.722222,16.518948,NaN
1997-01-06,NaN,NaN,0.135466,NaN,6.301713,2.076389,NaN,4.964448,8.534490,9.657921,...,NaN,7.350090,11.071292,6.612471,NaN,NaN,NaN,10.888889,16.916513,NaN
1997-01-07,NaN,NaN,0.132624,NaN,6.441064,2.076389,NaN,5.163640,8.832899,9.879935,...,NaN,7.389927,11.098971,6.734605,NaN,NaN,NaN,11.888889,16.976147,NaN
1997-01-08,NaN,NaN,0.133572,NaN,6.348165,2.000000,NaN,5.178962,8.395229,9.879935,...,NaN,7.469603,11.015944,6.647366,NaN,NaN,NaN,11.666667,17.234566,NaN
1997-01-09,NaN,NaN,0.134519,NaN,6.410101,1.986111,NaN,5.071704,8.156506,9.768930,...,NaN,7.489522,11.389590,6.629923,NaN,NaN,NaN,12.111111,17.214685,NaN


In [27]:
nulls = stock_prices.isnull().sum() / len(stock_prices)
null_pct = nulls[nulls > 0]
null_pct

A       0.108132
AAL     0.327227
ABBV    0.599494
ACN     0.170539
AEE     0.037534
          ...   
WYNN    0.217754
XYL     0.554066
YUM     0.026512
ZBH     0.171135
ZTS     0.602621
Length: 174, dtype: float64

In [28]:
stock_prices_nn = stock_prices.drop(null_pct.index, axis=1)
stock_prices_nn.isnull().sum().sum()

0

In [31]:
stock_prices_nn = stock_prices_nn.reindex(econ_df.index).ffill()

In [32]:
stock_prices_nn.isnull().sum()

AAPL    0
ABT     0
ACGL    0
ADBE    0
ADI     0
       ..
XEL     0
XOM     0
XRAY    0
ZBRA    0
ZION    0
Length: 329, dtype: int64

In [33]:
stock_prices_nn.shape

(1393, 329)

In [34]:
for i1, i2 in zip(stock_prices_nn.index, econ_df.index):
    assert i1 == i2

In [35]:
stock_prices_nn.to_csv("data/base/stock_prices.csv")